# Trying a simple CNN for Facial Recognition

### This was not the one I used because it was only on the facial data. 

In [1]:
import pandas as pd
import numpy as np
import keras 
from keras.models import Sequential
from keras.layers import Dense, Flatten, MaxPool2D, Conv2D, Dropout
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model
from sklearn.cross_validation import train_test_split
from IPython.display import display
import IPython 
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.
/Users/aubreybrowne/anaconda/envs/dl/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
jpgfiles = glob('/Users/aubreybrowne/Documents/dl_final_project/10k_faces_db/face_images/*.jpg')

In [3]:
X1 = []
for file in jpgfiles:
    image = load_img(file, target_size=(224, 224))
    image = img_to_array(image)
    X1.append(image)

In [4]:
X1 = np.array([x for x in X1])

In [5]:
myfiles = glob('/Users/aubreybrowne/Documents/dl_final_project/Pictures for DL project/*')

In [6]:
X2 = []
for file in myfiles:
    image = load_img(file, target_size=(224, 224))
    image = img_to_array(image)
    X2.append(image)

In [7]:
X2 = np.array([x for x in X2])

In [8]:
# Data Augmentation
datagen = ImageDataGenerator(rotation_range=1, #orginally 40 5 keras.preprocessing.image
    width_shift_range=0.05, #0.2 0.05
    height_shift_range=0.05, #0.2 0.05
    rescale=1./255,
    shear_range=0.2, # 0.2
    zoom_range=0.2,
    horizontal_flip=False, # originally True False
    fill_mode='nearest')

In [9]:
aug_img = []
image = datagen.flow(X2)
for x in X2:
    for _ in range(30):
        X = next(image)
        img = X[0]
        aug_img.append(img)

In [10]:
X3 = np.array([x for x in aug_img])

In [11]:
Xme = np.concatenate([X2, X3])

In [12]:
X = np.concatenate([X1, Xme])

In [13]:
y0 = np.zeros([10168])
y1 = np.ones([1240])
y = np.concatenate((y0, y1))
y

array([ 0.,  0.,  0., ...,  1.,  1.,  1.])

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, y)

In [15]:
y_train = keras.utils.to_categorical(y_train, num_classes=2)
y_test = keras.utils.to_categorical(y_test, num_classes=2)

In [18]:
# The model
model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=[224, 224, 3]))
#model.add(Conv2D(64, (10, 10), activation='relu'))
3model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 220, 220, 32)      2432      
_________________________________________________________________
dropout_3 (Dropout)          (None, 220, 220, 32)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1548800)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 3097602   
Total params: 3,100,034.0
Trainable params: 3,100,034.0
Non-trainable params: 0.0
_________________________________________________________________


In [19]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
8556/8556 [==============================] - 1181s - loss: 1.7767 - acc: 0.8894     

In [20]:
score = model.evaluate(x_test, 
                       y_test, 
                       verbose=False)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.66719431028
Test accuracy: 0.896563814867


In [21]:
import pickle
model.save('simpleCNN.h5')